# Q2: Data Cleaning

**Phase 3:** Data Cleaning & Preprocessing  
**Points: 9 points**

**Focus:** Handle missing data, outliers, validate data types, remove duplicates.

**Lecture Reference:** Lecture 11, Notebook 1 ([`11/demo/01_setup_exploration_cleaning.ipynb`](https://github.com/christopherseaman/datasci_217/blob/main/11/demo/01_setup_exploration_cleaning.ipynb)), Phase 3. Also see Lecture 05 (data cleaning).

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load data from Q1 (or directly from source)
df = pd.read_csv("data/beach_sensors.csv")

---

## Objective

Clean the dataset by handling missing data, outliers, validating data types, and removing duplicates.

**Time Series Note:** For time series data, forward-fill (`ffill()`) is often appropriate for missing values since sensor readings are continuous. However, you may choose other strategies based on your analysis.

---

## Required Artifacts

You must create exactly these 3 files in the `output/` directory:

### 1. `output/q2_cleaned_data.csv`
**Format:** CSV file
**Content:** Cleaned dataset with same structure as original (same columns)
**Requirements:**
- Same columns as original dataset
- Missing values handled (filled, dropped, or imputed)
- Outliers handled (removed, capped, or transformed)
- Data types validated and converted
- Duplicates removed
- **Sanity check:** Dataset should retain most rows after cleaning (at least 1,000 rows). If you're removing more than 50% of data, reconsider your strategy—imputation is usually preferable to dropping rows for this dataset.
- **No index column** (save with `index=False`)

### 2. `output/q2_cleaning_report.txt`
**Format:** Plain text file
**Content:** Detailed report of cleaning operations
**Required information:**
- Rows before cleaning: [number]
- Missing data handling method: [description]
  - Which columns had missing data
  - Method used (drop, forward-fill, impute, etc.)
  - Number of values handled
- Outlier handling: [description]
  - Detection method (IQR, z-scores, domain knowledge)
  - Which columns had outliers
  - Method used (remove, cap, transform)
  - Number of outliers handled
- Duplicates removed: [number]
- Data type conversions: [list any conversions]
- Rows after cleaning: [number]

**Example format:**
```
DATA CLEANING REPORT
====================

Rows before cleaning: 50000

Missing Data Handling:
- Water Temperature: 2500 missing values (5.0%)
  Method: Forward-fill (time series appropriate)
  Result: All missing values filled
  
- Air Temperature: 1500 missing values (3.0%)
  Method: Forward-fill, then median imputation for remaining
  Result: All missing values filled

Outlier Handling:
- Water Temperature: Detected 500 outliers using IQR method (3×IQR)
  Method: Capped at bounds [Q1 - 3×IQR, Q3 + 3×IQR]
  Bounds: [-5.2, 35.8]
  Result: 500 values capped

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]

Rows after cleaning: 50000
```

### 3. `output/q2_rows_cleaned.txt`
**Format:** Plain text file
**Content:** Single integer number (total rows after cleaning)
**Requirements:**
- Only the number, no text, no labels
- No whitespace before or after
- Example: `50000`

---

## Requirements Checklist

- [ ] Missing data handling strategy chosen and implemented
- [ ] Outliers detected and handled (IQR method, z-scores, or domain knowledge)
- [ ] Data types validated and converted
- [ ] Duplicates identified and removed
- [ ] Cleaning decisions documented in report
- [ ] All 3 required artifacts saved with exact filenames

---

## Your Approach

1. **Handle missing data** - Choose appropriate strategy (drop, forward-fill, impute) based on data characteristics
2. **Detect and handle outliers** - Use IQR method or z-scores; decide whether to remove, cap, or transform
3. **Validate data types** - Ensure numeric and datetime columns are properly typed
4. **Remove duplicates**
5. **Document and save** - Write detailed cleaning report explaining your decisions

---

## Decision Points

- **Missing data:** Should you drop rows, impute values, or forward-fill? Consider: How much data is missing? Is it random or systematic? For time series, forward-fill is often appropriate.
- **Outliers:** Are they errors or valid extreme values? Use IQR method or z-scores to detect, then decide: remove, cap, or transform. Document your reasoning.
- **Data types:** Are numeric columns actually numeric? Are datetime columns properly formatted? Convert as needed.

---

## Checkpoint

After Q2, you should have:
- [ ] Missing data handled
- [ ] Outliers addressed
- [ ] Data types validated
- [ ] Duplicates removed
- [ ] All 3 artifacts saved: `q2_cleaned_data.csv`, `q2_cleaning_report.txt`, `q2_rows_cleaned.txt`

---

**Next:** Continue to `q3_data_wrangling.md` for Data Wrangling.


## Handle Missing Data

In [2]:
missing_report = "Missing Data Handling:\n"

# Count missing values
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        missing_report += (
            f"- {col}: {missing_count} missing values ({missing_count / len(df):.1%})\n"
        )

# Fill rain-related variables with 0
rain_cols = [
    "Rain Intensity",
    "Total Rain",
    "Interval Rain",
    "Precipitation Type",
    "Wet Bulb Temperature",
]

for col in rain_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# Forward-fill + backward-fill all other columns
df_clean = df.ffill().bfill()

# Build final text
missing_report += (
    "\nMethod:\n"
    "- Rain-related variables filled with 0 (absence of precipitation)\n"
    "- All remaining variables forward-filled and backward-filled\n"
    "\nResult: All missing values filled\n"
)

print(missing_report)


Missing Data Handling:
- Air Temperature: 75 missing values (0.0%)
- Wet Bulb Temperature: 75930 missing values (38.7%)
- Rain Intensity: 75930 missing values (38.7%)
- Total Rain: 75930 missing values (38.7%)
- Precipitation Type: 75930 missing values (38.7%)
- Barometric Pressure: 146 missing values (0.1%)
- Heading: 75930 missing values (38.7%)

Method:
- Rain-related variables filled with 0 (absence of precipitation)
- All remaining variables forward-filled and backward-filled

Result: All missing values filled



## Handle Outliers

In [3]:
outlier_report = "Outlier Handling:\n"
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns

outlier_counts = {}  # store counts to summarize later

for col in numeric_cols:
    Q1, Q3 = df_clean[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR

    mask = (df_clean[col] < lower) | (df_clean[col] > upper)
    n_outliers = mask.sum()

    if n_outliers > 0:
        outlier_counts[col] = n_outliers
        df_clean[col] = df_clean[col].clip(lower, upper)

# Build clean list-style output
for col, n in outlier_counts.items():
    outlier_report += f"- {col}: {n} outliers detected (1.5×IQR)\n"

# Add method + result summary (like missing-data section)
outlier_report += (
    "\nMethod:\n"
    "- Outliers capped at column-specific 1.5×IQR bounds\n\n"
    "Result:\n"
    "- All detected outliers capped\n"
)

print(outlier_report)

Outlier Handling:
- Air Temperature: 97 outliers detected (1.5×IQR)
- Wet Bulb Temperature: 34 outliers detected (1.5×IQR)
- Humidity: 185 outliers detected (1.5×IQR)
- Rain Intensity: 4243 outliers detected (1.5×IQR)
- Interval Rain: 15850 outliers detected (1.5×IQR)
- Total Rain: 26697 outliers detected (1.5×IQR)
- Precipitation Type: 8414 outliers detected (1.5×IQR)
- Wind Speed: 12221 outliers detected (1.5×IQR)
- Maximum Wind Speed: 4024 outliers detected (1.5×IQR)
- Barometric Pressure: 4611 outliers detected (1.5×IQR)
- Solar Radiation: 29484 outliers detected (1.5×IQR)
- Heading: 27532 outliers detected (1.5×IQR)
- Battery Life: 6 outliers detected (1.5×IQR)

Method:
- Outliers capped at column-specific 1.5×IQR bounds

Result:
- All detected outliers capped



## Remove Duplicates & Data Type Conversions

In [4]:
initial_rows = len(df)

# Fix Negative Solar Radiation Values
if "Solar Radiation" in df_clean.columns:
    neg_sr = (df_clean["Solar Radiation"] < 0).sum()
    df_clean["Solar Radiation"] = df_clean["Solar Radiation"].clip(lower=0)
    negative_report = (
        "Negative Value Handling:\n"
        f"- Solar Radiation: {neg_sr} negative values corrected (set to 0)\n"
    )
else:
    neg_sr = 0
    negative_report = (
        "Negative Value Handling:\n- No negative Solar Radiation values detected\n"
    )

# Remove Duplicates
duplicates = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()
duplicate_report = f"Duplicates Removed: {duplicates}\n"

# Data Type Conversions
conversion_report = "Data Type Conversions:\n"
if "Measurement Timestamp" in df_clean.columns:
    df_clean["Measurement Timestamp"] = pd.to_datetime(
        df_clean["Measurement Timestamp"]
    )
    conversion_report += "- Measurement Timestamp: Converted to datetime64[ns]"

final_rows = len(df_clean)

print(negative_report)
print(duplicate_report)
print(conversion_report)
print(f"Rows Before Cleaning: {initial_rows}")
print(f"Rows After Cleaning: {final_rows}")

Negative Value Handling:
- Solar Radiation: 13425 negative values corrected (set to 0)

Duplicates Removed: 0

Data Type Conversions:
- Measurement Timestamp: Converted to datetime64[ns]
Rows Before Cleaning: 196279
Rows After Cleaning: 196279


## Save Artifacts

In [5]:
# 1. Cleaned Data
df_clean.to_csv("output/q2_cleaned_data.csv", index=False)

# 2. Cleaning Report
report_content = f"""
DATA CLEANING REPORT
====================
Rows Before Cleaning: {initial_rows}

{missing_report}
{negative_report}
{outlier_report}
{duplicate_report}
{conversion_report}

Rows After Cleaning: {final_rows}
"""

with open("output/q2_cleaning_report.txt", "w") as f:
    f.write(report_content)

# 3. Rows Cleaned
with open("output/q2_rows_cleaned.txt", "w") as f:
    f.write(str(final_rows))

print("Artifacts created successfully!")

Artifacts created successfully!
